In [46]:
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.optimize import linear_sum_assignment

In [4]:
df=pd.read_csv('processed data.csv')
df.head()

,Bus,Route number,Institute,Driver_Lat,Driver_Lon,PP_Lat,PP_Lon,Dead km,Dead_KM_Haversine
0,KA53AC0634,1,Chanakya,12.958330,77.525384,12.958330,77.525384,0.0,0.000000
1,KA53AC0540,2,Chanakya,13.249466,77.708713,13.249466,77.708713,0.0,0.000000
2,KA439840,1,Gitam,13.017359,77.657409,13.010978,77.695193,6.0,4.154431
3,KA43A0245,2,Gitam,13.033098,77.553878,12.967800,77.536097,6.0,7.512091
4,KA439841,3,Gitam,13.099021,77.574234,13.034584,77.557486,8.0,7.391135


In [19]:
base_url = "http://localhost:5000/route/v1/driving"
forward_dists, reverse_dists = [], []

In [39]:
# for _, row in df.iterrows():
#     src = f"{row['Driver_Lon']},{row['Driver_Lat']};{row['PP_Lon']},{row['PP_Lat']}"
    # rev = f"{row['PP_Lon']},{row['PP_Lat']};{row['Driver_Lon']},{row['Driver_Lat']}"
    # try:
    #     f_resp = requests.get(f"{base_url}/{src}?overview=false")
    #     forward = f_resp.json()['routes'][0]['distance'] / 1000 if f_resp.ok else None
    # except:
    #     forward = None

    # try:
    #     r_resp = requests.get(f"{base_url}/{rev}?overview=false")
    #     reverse = r_resp.json()['routes'][0]['distance'] / 1000 if r_resp.ok else None
    # except:
    #     reverse = None

    # forward_dists.append(forward)
    # reverse_dists.append(reverse)
    # time.sleep(0.2)

    # df['distance_1_to_2'] = forward_dists
    # df['distance_2_to_1'] = reverse_dists

def osrm_dist(lon1, lat1, lon2, lat2):
    src=f'{lon1},{lat1};{lon2},{lat2}'
    url=f"http://localhost:5000/route/v1/driving/{src}?overview=false"
    # res=requests.get(f"http://localhost:5000/route/v1/driving/{src}?overview=false")
    # # forward = res.json()['routes'][0]['distance'] / 1000
    # print(res.json())
    # forward=res.json()['routes'][0]['distance'] / 1000
    # print(res.json()['routes'])
    # return forward
    # url = f"http://localhost:5000/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    time.sleep(0.2)
    try:
        res = requests.get(url).json()
        return res['routes'][0]['distance'] / 1000  # convert meters to km
    except Exception as e:
        print(f"OSRM error from ({lat1},{lon1}) to ({lat2},{lon2}): {e}")
        return np.nan

# print(osrm_dist(77.65740947,13.0173592,77.63069568,13.03201432))
# df['OS_Distance'] = df.apply(
#     lambda row: osrm_dist(row['Driver_Lon'], row['Driver_Lat'], row['PP_Lon'], row['PP_Lat']),
#     axis=1
# )



In [36]:
df.head()

,Bus,Route number,Institute,Driver_Lat,Driver_Lon,PP_Lat,PP_Lon,Dead km,Dead_KM_Haversine
0,KA53AC0634,1,Chanakya,12.958330,77.525384,12.958330,77.525384,0.0,0.000000
1,KA53AC0540,2,Chanakya,13.249466,77.708713,13.249466,77.708713,0.0,0.000000
2,KA439840,1,Gitam,13.017359,77.657409,13.010978,77.695193,6.0,4.154431
3,KA43A0245,2,Gitam,13.033098,77.553878,12.967800,77.536097,6.0,7.512091
4,KA439841,3,Gitam,13.099021,77.574234,13.034584,77.557486,8.0,7.391135


In [18]:
buses = df['Bus'].unique()
dist_matrix = pd.DataFrame('', index=buses, columns=buses)

np.fill_diagonal(dist_matrix.values, 0)
dist_matrix.head()

,KA53AC0634,KA53AC0540,KA439840,KA43A0245,KA439841,KA439842,KA53B6963,KA53AC0345,KA51B9664,KA51AJ8298,...,KA07A2722,KA51B9235,KA53B1064,KA53D0153,KA53B1072,KA53B0960,KA53B0958,KA53B0952,KA53B1075,KA53B1061
KA53AC0634,0,,,,,,,,,,...,,,,,,,,,,
KA53AC0540,,0,,,,,,,,,...,,,,,,,,,,
KA439840,,,0,,,,,,,,...,,,,,,,,,,
KA43A0245,,,,0,,,,,,,...,,,,,,,,,,
KA439841,,,,,0,,,,,,...,,,,,,,,,,


In [12]:
# dist_matrix[df['Bus']]=''
dist_matrix.head()

,Bus,KA53AC0634,KA53AC0540,KA439840,KA43A0245,KA439841,KA439842,KA53B6963,KA53AC0345,KA51B9664,...,KA07A2722,KA51B9235,KA53B1064,KA53D0153,KA53B1072,KA53B0960,KA53B0958,KA53B0952,KA53B1075,KA53B1061
0,KA53AC0634,,,,,,,,,,...,,,,,,,,,,
1,KA53AC0540,,,,,,,,,,...,,,,,,,,,,
2,KA439840,,,,,,,,,,...,,,,,,,,,,
3,KA43A0245,,,,,,,,,,...,,,,,,,,,,
4,KA439841,,,,,,,,,,...,,,,,,,,,,


In [38]:
bus_coords = df.set_index('Bus')[['Driver_Lat', 'Driver_Lon']].to_dict('index')
buses = list(bus_coords.keys())

In [40]:
print(bus_coords)

{'KA53AC0634': {'Driver_Lat': 12.95832978, 'Driver_Lon': 77.52538372}, 'KA53AC0540': {'Driver_Lat': 13.24946574, 'Driver_Lon': 77.70871305}, 'KA439840': {'Driver_Lat': 13.0173592, 'Driver_Lon': 77.65740947}, 'KA43A0245': {'Driver_Lat': 13.03309838, 'Driver_Lon': 77.55387848}, 'KA439841': {'Driver_Lat': 13.09902109, 'Driver_Lon': 77.57423437}, 'KA439842': {'Driver_Lat': 13.0173592, 'Driver_Lon': 77.65740947}, 'KA53B6963': {'Driver_Lat': 13.09902109, 'Driver_Lon': 77.57423437}, 'KA53AC0345': {'Driver_Lat': 13.00814885, 'Driver_Lon': 77.48480867}, 'KA51B9664': {'Driver_Lat': 13.30939173, 'Driver_Lon': 77.53686262}, 'KA51AJ8298': {'Driver_Lat': 13.00877634, 'Driver_Lon': 77.69236841}, 'KA53AC0344': {'Driver_Lat': 12.93474, 'Driver_Lon': 77.690393}, 'KA51AJ8291': {'Driver_Lat': 12.98532739, 'Driver_Lon': 77.60170877}, 'KA51AJ8295': {'Driver_Lat': 12.98532739, 'Driver_Lon': 77.60170877}, 'KA51AJ8296': {'Driver_Lat': 13.03006532, 'Driver_Lon': 77.5445838}, 'KA53AB4252': {'Driver_Lat': 13.0990

In [43]:
for driver_bus in buses:
    driver_lat, driver_lon = bus_coords[driver_bus]['Driver_Lat'], bus_coords[driver_bus]['Driver_Lon']
    for pickup_bus in buses:
        pickup_lat, pickup_lon = bus_coords[pickup_bus]['Driver_Lat'], bus_coords[pickup_bus]['Driver_Lon']
        if driver_bus == pickup_bus:
            dist = 0.0
        else:
            dist = osrm_dist(driver_lon, driver_lat, pickup_lon, pickup_lat)
        dist_matrix.loc[driver_bus, pickup_bus] = dist
dist_matrix.head()

,KA53AC0634,KA53AC0540,KA439840,KA43A0245,KA439841,KA439842,KA53B6963,KA53AC0345,KA51B9664,KA51AJ8298,...,KA07A2722,KA51B9235,KA53B1064,KA53D0153,KA53B1072,KA53B0960,KA53B0958,KA53B0952,KA53B1075,KA53B1061
KA53AC0634,0.0,42.8617,19.866,12.2339,21.3237,19.866,21.3237,10.3967,47.6051,22.79,...,27.1815,27.1815,13.9668,22.8982,43.1339,11.3259,11.3259,19.6547,25.7264,4.536
KA53AC0540,43.2934,0.0,36.4976,33.0596,25.2427,36.4976,25.2427,42.501,21.8915,41.5439,...,23.9535,23.9535,34.8994,41.5466,37.8218,34.8892,34.8892,45.9763,43.6484,41.0971
KA439840,19.6618,34.938,0.0,16.4823,18.5213,0.0,18.5213,25.1978,41.7127,5.2916,...,20.7802,20.7802,19.8407,5.3999,44.2413,17.1998,17.1998,7.3898,9.7324,21.03
KA43A0245,11.8604,33.165,16.6553,0.0,10.1405,16.6553,10.1405,15.2016,36.422,18.6028,...,17.4848,17.4848,7.1505,18.7111,33.4648,3.939,3.939,16.8065,22.8782,10.1468
KA439841,21.4728,24.4306,18.8201,10.483,0.0,18.8201,0.0,18.4805,26.2815,21.6831,...,8.7286,8.7286,10.8789,21.7914,28.81,12.3124,12.3124,24.0781,26.1022,18.5202


In [49]:
dist_matrix.to_csv('phase1_distance_matrix.csv', index=False)
copy_matrix=dist_matrix.copy()
copy_matrix[copy_matrix==0]=np.inf
print(copy_matrix)

           KA53AC0634 KA53AC0540 KA439840 KA43A0245 KA439841 KA439842  \
KA53AC0634        inf    42.8617   19.866   12.2339  21.3237   19.866   
KA53AC0540    43.2934        inf  36.4976   33.0596  25.2427  36.4976   
KA439840      19.6618     34.938      inf   16.4823  18.5213      inf   
KA43A0245     11.8604     33.165  16.6553       inf  10.1405  16.6553   
KA439841      21.4728    24.4306  18.8201    10.483      inf  18.8201   
KA439842      19.6618     34.938      inf   16.4823  18.5213      inf   
KA53B6963     21.4728    24.4306  18.8201    10.483      inf  18.8201   
KA53AC0345    10.4882    42.2138  25.6833   14.4307  17.7831  25.6833   
KA51B9664     47.7179    21.9102  42.7846   36.7282  27.4673  42.7846   
KA51AJ8298    22.3961    34.8109   6.2836   20.0776  24.6422   6.2836   
KA53AC0344    21.2822    51.2563  16.5571   25.5235  34.7007  16.5571   
KA51AJ8291    11.2516    35.3755   8.7277    9.9234   17.566   8.7277   
KA51AJ8295    11.2516    35.3755   8.7277    9.9234

In [53]:
optimized_assignments=linear_sum_assignment(copy_matrix)

In [54]:
print(optimized_assignments)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52], dtype=int64), array([24, 42, 16, 13, 37, 30, 38, 26, 47, 46, 23, 17, 50, 48, 39, 51,  2,
       11, 22, 52, 31, 41,  4, 10,  0, 43,  7, 44, 29, 28, 12, 20, 33, 21,
       36, 40, 32, 18, 34,  6, 27, 25, 35,  1, 14, 49,  9,  8, 45,  3,  5,
       15, 19], dtype=int64))


In [55]:
row_pickup, col_driver=optimized_assignments

In [56]:
assignments = list(zip(dist_matrix.index[row_pickup], dist_matrix.columns[col_driver]))

In [57]:
print(assignments)

[('KA53AC0634', 'KA53D4606'), ('KA53AC0540', 'KA53B0163'), ('KA439840', 'KA50A5968'), ('KA43A0245', 'KA51AJ8296'), ('KA439841', 'KA01AC4662'), ('KA439842', 'KA53D4609'), ('KA53B6963', 'KA53D0155'), ('KA53AC0345', 'KA53D5165'), ('KA51B9664', 'KA53B1072'), ('KA51AJ8298', 'KA53D0153'), ('KA53AC0344', 'KA53D8420'), ('KA51AJ8291', 'KA53AB1811'), ('KA51AJ8295', 'KA53B0952'), ('KA51AJ8296', 'KA53B0960'), ('KA53AB4252', 'KA07A9091'), ('KA53AB1810', 'KA53B1075'), ('KA50A5968', 'KA439840'), ('KA53AB1811', 'KA51AJ8291'), ('KA53D5405', 'KA53B6965'), ('KA53AB1808', 'KA53B1061'), ('KA53D5162', 'KA53D2719'), ('KA53D0166', 'KA07A9134'), ('KA53B6965', 'KA439841'), ('KA53D8420', 'KA53AC0344'), ('KA53D4606', 'KA53AC0634'), ('KA53D8419', 'KA07A2722'), ('KA53D5165', 'KA53AC0345'), ('KA53D0158', 'KA51B9235'), ('KA07A9086', 'KA53D2958'), ('KA53D2958', 'KA07A9086'), ('KA53D4609', 'KA51AJ8295'), ('KA53D2719', 'KA53D5162'), ('KA53B6967', 'KA53B6964'), ('KA53B6964', 'KA53D0166'), ('KA07A9089', 'KA53D0162'), ('KA

In [59]:
assignments_df = pd.DataFrame({
    'From_Bus': dist_matrix.index[row_pickup],
    'To_Bus': dist_matrix.columns[col_driver],
    'Distance_km': dist_matrix.values[row_pickup, col_driver]
})

In [61]:
assignments_df.to_csv('phase1_demo assignments.csv', index=False)